In [1]:
import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import config
from dataset import INAOEDataset, DFUData

In [2]:
# Estimated Temperature (ET)
def estimated_temperature(hist: np.ndarray, cluster: np.ndarray):
    argmax = np.argmax(hist)
    if argmax < len(cluster) - 1:
        et_hist = hist[[argmax-1, argmax, argmax+1]]
        et_cluster = cluster[[argmax-1, argmax, argmax+1]]
    else:
        et_hist = hist[[argmax-1, argmax]]
        et_cluster = cluster[[argmax-1, argmax]]
    # et_cluster = cluster[[argmax-1, argmax, argmax+2]]
    return np.sum(et_hist * et_cluster) / np.sum(et_hist)

# Estimated Temperature difference (ETD)
def estimated_temperature_difference(left_et_angiosome, right_et_angiosome):
    return np.abs(left_et_angiosome - right_et_angiosome)

# Hot Pot Estimator
def hot_spot_estimator(hist:np.ndarray, cluster:np.ndarray):
    argmax = np.argmax(hist)
    et = estimated_temperature(hist, cluster)
    return np.abs(cluster[argmax] - et)


In [20]:
dm_group = True
save_path = 'data/DM' if dm_group else 'data/CG'

dataset = INAOEDataset(config.INAOE_DATASET_DIR, dm_group=dm_group)

In [21]:
from scipy.stats import skew, kurtosis

def get_metrics(dataset, data:DFUData):
    subjects = dataset.get_subjects_names()
    mean_data = []
    std_data = []
    max_data = []
    min_data = []
    skew_data = []
    kurtosis_data = []
    expected_temerature_data = []
    hot_spot_data = []
    metrics = []
    nrt_c0, nrt_c1, nrt_c2, nrt_c3, nrt_c4, nrt_c5, nrt_c6, nrt_c7 = [], [], [], [], [], [], [], []

    fulldata = None
    for idx in range(len(dataset)):
        subject_data = dataset.get_subject(idx, data).to_numpy()
        subject_data = subject_data[subject_data>18] # Remove background, temperatures lower than 18 are considered outliers!
        mean_data.append(subject_data.mean())
        std_data.append(subject_data.std())
        max_data.append(subject_data.max())
        min_data.append(subject_data.min())
        skew_data.append(skew(subject_data))
        kurtosis_data.append(kurtosis(subject_data))

        hist, _ = np.histogram(subject_data, bins=config.EXPECTED_VALUE_INTERVAL)
    
        # NRTClass
        nrt_class = hist / hist.sum()
        nrt_c0.append(nrt_class[0])
        nrt_c1.append(nrt_class[1])
        nrt_c2.append(nrt_class[2])
        nrt_c3.append(nrt_class[3])
        nrt_c4.append(nrt_class[4])
        nrt_c5.append(nrt_class[5])
        nrt_c6.append(nrt_class[6])
        nrt_c7.append(nrt_class[7])
    
        # Expected Temperature (ET) and Hot Spot Estimator
        expected_temerature_data.append(estimated_temperature(hist, config.EXPECTED_VALUE_CLUSTER))
        hot_spot_data.append(hot_spot_estimator(hist, config.EXPECTED_VALUE_CLUSTER))

    metrics = {'Subject': subjects,
        '{}_mean'.format(data.name): mean_data, 
        '{}_std'.format(data.name): std_data,
        '{}_min'.format(data.name): min_data,
        '{}_max'.format(data.name): max_data,
        '{}_skew'.format(data.name): skew_data,
        '{}_kurtosis'.format(data.name): kurtosis_data,
        '{}_NRT_C0'.format(data.name): nrt_c0,
        '{}_NRT_C1'.format(data.name): nrt_c1,
        '{}_NRT_C2'.format(data.name): nrt_c2,
        '{}_NRT_C3'.format(data.name): nrt_c3,
        '{}_NRT_C4'.format(data.name): nrt_c4,
        '{}_NRT_C5'.format(data.name): nrt_c5,
        '{}_NRT_C6'.format(data.name): nrt_c6,
        '{}_NRT_C7'.format(data.name): nrt_c7,
        '{}_HSE'.format(data.name): hot_spot_data,
        '{}_ET'.format(data.name): expected_temerature_data,
    }

    return pd.DataFrame(metrics)

# MPA

In [22]:
l_mpa_metrics = get_metrics(dataset, DFUData.L_MPA)
r_mpa_metrics = get_metrics(dataset, DFUData.R_MPA)

mpa_metrics = pd.merge(l_mpa_metrics, r_mpa_metrics, on='Subject')
ETD = estimated_temperature_difference(mpa_metrics['L_MPA_ET'], mpa_metrics['R_MPA_ET'])
mpa_metrics['MPA_ETD'] = ETD

mpa_metrics.to_csv(os.path.join(save_path, 'MPA/MPA_metrics.csv'), index=False)

# LCA

In [23]:
l_lca_metrics = get_metrics(dataset, DFUData.L_LCA)
r_lca_metrics = get_metrics(dataset, DFUData.R_LCA)

lca_metrics = pd.merge(l_lca_metrics, r_lca_metrics, on='Subject')
ETD = estimated_temperature_difference(lca_metrics['L_LCA_ET'], lca_metrics['R_LCA_ET'])
lca_metrics['LCA_ETD'] = ETD

lca_metrics.to_csv(os.path.join(save_path, 'LCA/LCA_metrics.csv'), index=False)

# LPA

In [24]:
l_lpa_metrics = get_metrics(dataset, DFUData.L_LPA)
r_lpa_metrics = get_metrics(dataset, DFUData.R_LPA)

lpa_metrics = pd.merge(l_lpa_metrics, r_lpa_metrics, on='Subject')
ETD = estimated_temperature_difference(lpa_metrics['L_LPA_ET'], lpa_metrics['R_LPA_ET'])
lpa_metrics['LPA_ETD'] = ETD

lpa_metrics.to_csv(os.path.join(save_path, 'LPA/LPA_metrics.csv'), index=False)

# MCA

In [25]:
l_mca_metrics = get_metrics(dataset, DFUData.L_MCA)
r_mca_metrics = get_metrics(dataset, DFUData.R_MCA)

mca_metrics = pd.merge(l_mca_metrics, r_mca_metrics, on='Subject')
ETD = estimated_temperature_difference(mca_metrics['L_MCA_ET'], mca_metrics['R_MCA_ET'])
mca_metrics['MCA_ETD'] = ETD

mca_metrics.to_csv(os.path.join(save_path, 'MCA/MCA_metrics.csv'), index=False)

# Full Foot

In [26]:
l_metrics = get_metrics(dataset, DFUData.L)
r_metrics = get_metrics(dataset, DFUData.R)

full_metrics = pd.merge(l_metrics, r_metrics, on='Subject')
ETD = estimated_temperature_difference(full_metrics['L_ET'], full_metrics['R_ET'])
full_metrics['Foot_ETD'] = ETD

full_metrics.to_csv(os.path.join(save_path, 'Full/Full_metrics.csv'), index=False)

In [27]:
full_metrics

,Subject,L_mean,L_std,L_min,L_max,L_skew,L_kurtosis,L_NRT_C0,L_NRT_C1,L_NRT_C2,...,R_NRT_C1,R_NRT_C2,R_NRT_C3,R_NRT_C4,R_NRT_C5,R_NRT_C6,R_NRT_C7,R_HSE,R_ET,Foot_ETD
0,DM001_M,32.094376,0.694608,27.927000,34.457001,0.223109,0.213512,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.019760,0.251906,0.425556,32.925556,1.055676
1,DM002_M,26.333252,1.337137,19.427999,29.011000,-0.824748,1.789838,0.007384,0.287168,0.428457,...,0.178886,0.264321,0.353470,0.150538,0.043793,0.000000,0.000000,0.148092,27.351908,1.462198
2,DM003_F,27.186190,0.930826,24.704000,29.382000,-0.069757,-0.306776,0.000000,0.111064,0.274344,...,0.064557,0.094659,0.294864,0.351479,0.175612,0.018829,0.000000,0.145083,28.354917,0.983584
3,DM004_M,30.705494,1.082484,25.792999,32.713001,-0.858052,0.455516,0.000000,0.000228,0.001934,...,0.000000,0.000000,0.000000,0.000249,0.001243,0.034307,0.313735,0.197370,32.697370,1.484857
4,DM005_M,25.356159,1.260719,20.356001,28.465000,-0.484952,-0.270005,0.004001,0.661450,0.260414,...,0.881517,0.096841,0.000109,0.000000,0.000000,0.000000,0.000000,0.155990,24.155990,0.529889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,DM118_M,32.274334,1.515720,28.299999,36.027000,0.536526,-0.549614,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000186,0.013564,0.128205,0.339651,0.263657,0.185166,30.685166,0.781885
118,DM119_F,33.685417,0.568763,28.777000,35.181000,-1.486008,4.764882,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000495,0.009080,0.078422,0.906499,34.093501,0.674798
119,DM120_F,32.283695,1.337566,29.354000,35.240002,-0.023549,-1.215146,0.000000,0.000000,0.000000,...,0.000000,0.000179,0.000179,0.000000,0.001793,0.039627,0.161018,0.820453,33.320453,0.855136
120,DM121_F,33.131172,1.065738,27.691000,35.599998,-0.287688,0.159925,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000285,0.004280,0.015121,0.644645,34.355355,0.417825


# Merge

In [28]:
from functools import reduce 

feature_dataset_df = reduce(lambda left,right: pd.merge(left,right,on='Subject'), [mpa_metrics, lca_metrics, lpa_metrics, mca_metrics, full_metrics])

group = 'dm' if dm_group else 'cg'
feature_dataset_df.to_csv(os.path.join(save_path, '{}_metrics.csv'.format(group)), index=False)

# TCI

In [29]:
def temperature_cg_values(angiosomes: DFUData):
    '''
        Returns the control temperature for the given angiosomes based on the paper
    '''
    data = [-1, -1, 26.1, 26.1, 25.7, 25.7, 26.4, 26.4, 25.8, 25.8]
    if isinstance(angiosomes, list):
        return np.array(list(map(lambda x: data[x.value], angiosomes)))
    else:
        return data[angiosomes.value]

def thermal_change_index(features_df: pd.DataFrame):
    feet_angiosomes = [ [DFUData.L_LCA, DFUData.L_LPA, DFUData.L_MCA, DFUData.L_MPA],
                        [DFUData.R_LCA, DFUData.R_LPA, DFUData.R_MCA, DFUData.R_MPA] ]

    tci = []
    for foot_angiosomes in feet_angiosomes:
        columns = list(map(lambda x: '{}_mean'.format(x.name), foot_angiosomes))
        control_temperature = temperature_cg_values(foot_angiosomes)
        tci.append((np.abs(control_temperature - features_df[columns].to_numpy())).mean(axis=1))
    
    return tci

left_tci, right_tci = thermal_change_index(feature_dataset_df)

feature_dataset_df['L_TCI'] = left_tci
feature_dataset_df['R_TCI'] = right_tci    

group = 'dm' if dm_group else 'cg'
feature_dataset_df.to_csv(os.path.join(save_path, '{}_metrics.csv'.format(group)), index=False)

In [30]:
feature_dataset_df

,Subject,L_MPA_mean,L_MPA_std,L_MPA_min,L_MPA_max,L_MPA_skew,L_MPA_kurtosis,L_MPA_NRT_C0,L_MPA_NRT_C1,L_MPA_NRT_C2,...,R_NRT_C3,R_NRT_C4,R_NRT_C5,R_NRT_C6,R_NRT_C7,R_HSE,R_ET,Foot_ETD,L_TCI,R_TCI
0,DM001_M,32.394947,0.570629,30.261000,33.824001,-0.020788,-0.788161,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.019760,0.251906,0.425556,32.925556,1.055676,6.030594,6.482348
1,DM002_M,26.451084,1.454454,20.191000,29.011000,-0.797828,1.176556,0.010407,0.256386,0.387890,...,0.353470,0.150538,0.043793,0.000000,0.000000,0.148092,27.351908,1.462198,0.379689,0.926383
2,DM003_F,27.375771,0.596349,25.296000,29.238001,0.274395,1.542174,0.000000,0.016458,0.169279,...,0.294864,0.351479,0.175612,0.018829,0.000000,0.145083,28.354917,0.983584,1.297726,2.138911
3,DM004_M,30.718662,0.758783,28.069000,32.386002,-0.376139,-0.009581,0.000000,0.000000,0.000000,...,0.000000,0.000249,0.001243,0.034307,0.313735,0.197370,32.697370,1.484857,4.624053,6.169223
4,DM005_M,25.574556,0.492072,24.082001,28.465000,0.201779,1.724877,0.000000,0.828018,0.169481,...,0.000109,0.000000,0.000000,0.000000,0.000000,0.155990,24.155990,0.529889,0.866794,1.635378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,DM118_M,33.918480,1.387068,29.327999,36.027000,-0.720069,-0.170135,0.000000,0.000000,0.000000,...,0.000186,0.013564,0.128205,0.339651,0.263657,0.185166,30.685166,0.781885,6.293808,5.271137
118,DM119_F,33.731735,0.470749,31.774000,35.167000,-0.864708,0.777110,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000495,0.009080,0.078422,0.906499,34.093501,0.674798,7.678588,6.962305
119,DM120_F,33.344475,0.692031,29.847000,34.285000,-1.625465,3.438301,0.000000,0.000000,0.000000,...,0.000179,0.000000,0.001793,0.039627,0.161018,0.820453,33.320453,0.855136,6.094967,6.774374
120,DM121_F,33.488728,0.910405,30.636999,35.029999,-0.609535,-0.472014,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000285,0.004280,0.015121,0.644645,34.355355,0.417825,6.995157,7.444658


# Dataset

In [31]:
save_dir = 'data'
cg_df = pd.read_csv(os.path.join(save_dir, 'CG/cg_metrics.csv'))
dm_df = pd.read_csv(os.path.join(save_dir, 'DM/dm_metrics.csv'))

pd.concat([cg_df,dm_df], axis=0).to_csv(os.path.join(save_dir, 'dfu_features_dataset.csv'), index=False)